Phan1. Giới thiệu và cài đặt

In [1]:
# Cài đặt spaCy
!pip install -U spacy
# Tải về mô hình tiếng Anh (kích thước trung bình, có đủ thông tin cho parsing)
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 10.0 MB/s  0:00:03m0:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


phần 2 . Phân tích câu và trực quan hóa

In [2]:
import spacy
from spacy import displacy

# Tải mô hình tiếng Anh đã cài đặt
# Sử dụng en_core_web_md vì nó chứa các vector từ và cây cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")
# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."
# Phân tích câu với pipeline của spaCy
doc = nlp(text)

In [3]:
displacy.serve(doc  , style = 'dep')

/home/manh/code/nlp/.venv/lib/python3.10/site-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [08/Dec/2025 15:44:18] "GET / HTTP/1.1" 200 7538


Shutting down server on port 5000.


Câu hỏi 1 : Từ nào là từ gốc của ROOT.
TỪ gốc của ROOT ở đây là JUMPS
Câu hỏi 2 : jumps có những từ phụ thuộc (dependent) nào? Các quan hệ đó là gì?
Các từ phụ thuộc của JUMPS là "FOX" và "OVER".
Câu hỏi 3 : fox là head  của những từ nào?
Fox là head của det (Determiner - Mạo từ/Từ hạn định) , amod (Adjectival Modifier - Tính từ bổ nghĩa) , amod (Adjectival Modifier - Tính từ bổ nghĩa).

Truy cập các thành phần trong cây phụ thuộc .

In [10]:
# Lấy một câu khác để phân tích
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)
# In ra thông tin của từng token
print(f"{'TEXT': <12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
# Trích xuất các thuộc tính
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


Phần 4. Duyệt cây phụ thuộc để trích xuất thông tin.

In [15]:
# bài toán 1 . TÌm của ngữ và tân ngữ của một động từ.

text = "the cat chased the mouse and the dog watched them"
doc = nlp(text)
for token in doc :
    if token.pos_ == "VERB" :
        verb = token.text
        subject = ""
        obj = ""
    # tim chu ngu (nsubj) và tân ngữ (dobj) trong các con của động từ.
        for child in token.children:
            if child.dep_ == "nsubj" :
                subject = child.text
            if child.dep_ == "dobj" :
                obj = child.text
        if subject and obj :
            print(f"Found Triplet: ({subject}, {verb}, {obj})")



Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


In [20]:
# tìm tính từ bổ nghĩa cho danh từ.

text = "the big, fluffy white cat is sleeping on the warm mat."

doc = nlp(text)

for token in doc:
    if token.pos_ == "NOUN" :
        adjectives = []
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")



Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


Phân 5 . Bài tập tự luyện

In [22]:
# Bài 1: TÌm động từ chính của câu.

def find_main_verb(doc) :
    main_verbs = None
    for token in doc:
        if token.dep_ == "ROOT":
            main_verbs = token.text
            break
    print(f"Main Verb: {main_verbs}")
sentence = "The manager decided to cancel the meeting because it was raining."
doc = nlp(sentence)
find_main_verb(doc)


Main Verb: decided


In [30]:
# bài 2 : trích xuất các cụm danh từ (NOUN chunks)

import spacy


def get_noun_chunks_manual(doc):
    print(f"{'HEAD NOUN':<15} {'DETECTED CHUNK'}")
    print("-" * 40)

    for token in doc:
        # Chỉ xét nếu từ đó là Danh từ (NOUN) hoặc Đại từ (PROPN - tên riêng)
        if token.pos_ in ["NOUN", "PROPN"]:

            # KIỂM TRA QUAN TRỌNG:
            # Chúng ta chỉ muốn in ra cụm danh từ "lớn nhất".
            # Nếu danh từ này lại phụ thuộc vào một danh từ khác (ví dụ: "manager" trong "credit manager")
            # thì ta thường bỏ qua để tránh trùng lặp.
            if token.dep_ == "compound":
                continue

            # --- SỬ DỤNG SUBTREE ---
            # Lấy tất cả các từ trong nhánh của danh từ này theo đúng thứ tự
            chunk_words = [t.text for t in token.subtree]
            chunk_text = " ".join(chunk_words)

            print(f"{token.text:<15} {chunk_text}")

# Test với câu của bạn
sentence = "The manager decided to cancel the meeting because it was raining."
doc = nlp(sentence)

print("--- KẾT QUẢ CODE THỦ CÔNG ---")
get_noun_chunks_manual(doc)

print("\n--- KẾT QUẢ CỦA SPACY (ĐỂ SO SÁNH) ---")
for chunk in doc.noun_chunks:
    print(f"{chunk.root.text:<15} {chunk.text}")

--- KẾT QUẢ CODE THỦ CÔNG ---
HEAD NOUN       DETECTED CHUNK
----------------------------------------
manager         The manager
meeting         the meeting

--- KẾT QUẢ CỦA SPACY (ĐỂ SO SÁNH) ---
manager         The manager
meeting         the meeting
it              it


In [31]:
# bài 3 : Tìm đường đi ngắn nhất trong cây.

import spacy


def get_path_to_root(token):
    """
    Tìm đường đi từ token hiện tại lên tới ROOT.
    """
    path = []

    # Duyệt ngược lên trên cho đến khi gặp ROOT
    # (Tại ROOT, token.head sẽ trỏ vào chính nó)
    while token.head != token:
        path.append(token) # Thêm từ hiện tại vào đường đi
        token = token.head # Nhảy lên từ cha

    path.append(token) # Đừng quên thêm từ ROOT cuối cùng vào
    return path

# --- CHẠY THỬ NGHIỆM ---
sentence = "The quick brown fox jumps over the lazy dog."
doc = nlp(sentence)

# Giả sử chúng ta chọn từ "lazy" (từ thứ 7, index 6) để tìm đường về gốc
# Cấu trúc mong đợi: lazy -> dog -> over -> jumps (ROOT)
selected_token = doc[7] # Từ "dog"
# Hoặc chọn "lazy": selected_token = doc[6]

print(f"Đang tìm đường đi cho từ: '{selected_token.text}'")
path = get_path_to_root(selected_token)

# In kết quả đẹp mắt
print(f"Đường đi tìm được ({len(path)} bước):")
print("-" * 30)
for i, token in enumerate(path):
    # Vẽ mũi tên trừ bước cuối cùng
    arrow = "-->" if i < len(path) - 1 else "(ROOT)"
    print(f"{token.text:<10} [{token.dep_}] {arrow}")


Đang tìm đường đi cho từ: 'lazy'
Đường đi tìm được (4 bước):
------------------------------
lazy       [amod] -->
dog        [pobj] -->
over       [prep] -->
jumps      [ROOT] (ROOT)
